In [40]:
import itertools
import operator
import sys
import os
sys.path.append("../src")
from glob import glob
import pandas as pd
from neuraldb.dataset.base_reader import NeuralDatabaseDatasetReader
from neuraldb.dataset.generators.v0_4_database_reader import V4DatabaseSpecificReader

In [41]:
search_root = "/checkpoint/jth/job_staging/neuraldb_expts/experiment=*oracle*"
checkpoint_name = "best_em.ckpt"
files = glob("{}*/**/{}".format(search_root,checkpoint_name), recursive=True)


print(len(files))

40


In [42]:
def expand(idx,chunk):
  #if idx == 0:
  #  return ["job_id={}".format(chunk)]
  #elif idx == 1:
  #  return ["experiment={}".format(chunk)]
  if chunk.startswith("seed-"):
    return ["seed={}".format(chunk.replace("seed-",""))]
  elif "," in chunk:
    return chunk.split(",")
  elif "=" in chunk:
    return [chunk]
  return []

experiments = []
for file in files:
    chunks = file.replace(search_root,"").split("/")
    chunks = itertools.chain(*[expand(idx, chunk) for idx, chunk in enumerate(chunks)])

    data = {k:v for k,v in (chunk.split("=") for chunk in chunks)}
    data["file"] = file
    data['dir'] = os.path.dirname(file)
    experiments.append(data)




In [43]:
for exp in experiments:
    version = exp["version"]

    if version != "v0.5":
        continue

    module = "neuraldb.commands.t5train2" if "fusion" not in exp["file"] else "neuraldb.commands.t5train2_fusion"

    print("python -m {} "
          "--output_dir {} "
          "--model_name_or_path t5-base "
          "--do_predict "
          "--train_batch_size 8 "
          "--eval_batch_size 8 "
          "--n_gpu 2 "
          "--oracle "
          "--train_path={}/train_queries_last_{}.json "
          "--val_path={}/dev_queries_last_{}.json "
          "--test_path={}/dev_queries_last_{}.json "
          "--dataset_version={}".format(module, exp['dir'],version,exp['context'],version,exp['context'],version,exp['context'],version))



python -m neuraldb.commands.t5train2 --output_dir /checkpoint/jth/job_staging/neuraldb_expts/experiment=oracle_concat/dataset=ndb,model=t5-base,oracle=true,version=v0.5/lr=0.0005/context=50,filters=null,max_queries=null/seed-3 --model_name_or_path t5-base --do_predict --train_batch_size 8 --eval_batch_size 8 --n_gpu 2 --oracle --train_path=v0.5/train_queries_last_50.json --val_path=v0.5/dev_queries_last_50.json --test_path=v0.5/dev_queries_last_50.json --dataset_version=v0.5
python -m neuraldb.commands.t5train2 --output_dir /checkpoint/jth/job_staging/neuraldb_expts/experiment=oracle_concat/dataset=ndb,model=t5-base,oracle=true,version=v0.5/lr=0.0005/context=50,filters=null,max_queries=null/seed-1 --model_name_or_path t5-base --do_predict --train_batch_size 8 --eval_batch_size 8 --n_gpu 2 --oracle --train_path=v0.5/train_queries_last_50.json --val_path=v0.5/dev_queries_last_50.json --test_path=v0.5/dev_queries_last_50.json --dataset_version=v0.5
python -m neuraldb.commands.t5train2 --o